1\. Write a function that converts number representation (bin<->dec<->hex)

In [35]:
# Strign to convert into binary values
binConv = "01"
# Strign to convert into hexadecimal values
hexConv = "0123456789ABCDEF"

def DecToBin(Dec):
    # Result converted to binary
    binNum = ""
    while(Dec > 0):
        # Add at the beginning of the string the rest of the division
        binNum = binConv[Dec%2] + binNum
        # Divide the number
        Dec = Dec//2
    return binNum

def DecToHex(Dec):
    # Result converted to hexadecimal
    hexNum = ""
    while(Dec > 0):
        # Add at the beginning of the string the rest of the division
        hexNum = hexConv[Dec%16] + hexNum
        # Divide the number
        Dec = Dec//16
    return hexNum

def BinToDec(Bin):
    # Result converted to decimal
    dec = 0
    # Write in reverse order:
    # LSB is the first element in the string, MSB is the last
    Bin = Bin[::-1] #string[start:end:step]
    for index, value in enumerate(Bin):
        #dec = dec + int(value, 2)*2**index
        dec = dec + binConv.find(value[:])*2**index
    return dec

def BinToHex(Bin):
    return(DecToHex(BinToDec(Bin)))

def HexToDec(Bin):
    # Result converted to decimal
    dec = 0
    # Write in reverse order:
    # LSB is the first element in the string, MSB is the last
    Bin = Bin[::-1]
    for index, value in enumerate(Bin):
        dec = dec + hexConv.find(value[:])*16**index
    return dec

def HexToBin(Bin):
    return(DecToBin(HexToDec(Bin)))
    


a = 15
print("Decimal Number =", a)
print("Converted to Bin =", DecToBin(a))
print("Converted to Hex =", DecToHex(a))
print("\n --- \n")

b = "1101"
print("Binary Number =", b)
print("Converted to Dec =", BinToDec(b))
print("Converted to Hex =", BinToHex(b))
print("\n --- \n")

c = "10"
print("Hexadecimal Number =", c)
print("Converted to Dec =", HexToDec(c))
print("Converted to Bin =", HexToBin(c))        
    

Decimal Number = 15
Converted to Bin = 1111
Converted to Hex = F

 --- 

Binary Number = 1101
Converted to Dec = 13
Converted to Hex = D

 --- 

Hexadecimal Number = 10
Converted to Dec = 16
Converted to Bin = 10000


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [23]:
# TO DO !!!

def ConvertToFloat(st):
    sign = (st >> 31) & 1
    print(sign)
    exponent = ((st << 1) >> 24) | 1
    print(exponent - 127)
    mantissa = ((st << 9) >> 9) | 1
    print(1 + mantissa)
    print((1+mantissa)*2**(exponent-127))
    return

st = 118
ConvertToFloat(st)

0
-126
120
1.410593220986745e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [ ]:
a = 1
b = 2

while(a/2 != 0):
    a = a/2

'''
for b in range(2**310):
    try:
        b = b**2
    except OverflowError as err:
        print(err)
'''        
    

print(a)
print(b)

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
